## Load libraries

In [34]:
import pandas as pd
import numpy as np
import os
import glob
import preprocessing as preprocessing
import torch
print(torch.__version__)
from torch_geometric.data import Data

#Lets start at src location
if os.path.exists("./src"):
    os.chdir("./src")

config = {
    "counter_files_path"                : "../data/counters_temporal_data_2023-03-03T09-24-06/",
    "counters_nontemporal_aggregated"   : "../data/counters_non_temporal_aggregated_data.csv",
    "N_GRAPHS"                          : 30*24,
    "F_IN"                              : 7*24,
    "F_OUT"                             : 7*24,
    "target_col"                        : "Sum"
}

1.13.1


## Data preparation

In [20]:
import importlib
importlib.reload(preprocessing)

counters_df = pd.DataFrame()
for fname in glob.glob(config["counter_files_path"] + "*.csv"):
    counter_data = pd.read_csv(fname)
    counter_data = preprocessing.fill_gaps(counter_data)
    counter_data['Date'] = pd.to_datetime(counter_data['Date']) 
    counter_data.index = counter_data['Date']
    counter_data = counter_data.sort_index(ascending=False)
    # We don't need to work with all past data.
    # Select enough data points to extract N_GRAPHS with F_IN and F_OUT timepoints
    
    counter_data = counter_data.iloc[0:(config["F_IN"]+config["F_OUT"]+config["N_GRAPHS"]-1), :]
    counter_id = fname.split('/')[-1].split('.csv')[0]

    if counters_df.empty:
        counters_df = pd.DataFrame(counter_data[config['target_col']])
        counters_df.columns = [counter_id]
    else:
        columns = list(counters_df.columns) + [counter_id]
        counters_df = pd.concat([counters_df, counter_data[config['target_col']]], axis=1)
        counters_df.columns = columns 


#Prepare edge_index matrix
counters_aggregated = pd.read_csv(config['counters_nontemporal_aggregated'])
edge_index, n_node, num_edges = preprocessing.construct_edge_index(counters_aggregated)

In [ ]:
#Prepare matrices X [N_GRAPHS, F_IN, N_NODES] and Y [N_GRAPHS, F_OUT, N_NODES] 
graphs = []
for i in range(1, config["N_GRAPHS"]+1):
    
    g = Data()
    g.__num_nodes__ = n_node
    g.edge_index = edge_index

    train_test_chunk = counters_df.iloc[(-i-(config['F_IN']+config['F_OUT'])):(-i),:]
    print(i, train_test_chunk.shape, train_test_chunk.iloc[:config['F_IN'],:].to_numpy().shape)
    g.x = torch.FloatTensor(train_test_chunk.iloc[:config['F_IN'],:].to_numpy())
    g.y = torch.FloatTensor(train_test_chunk.iloc[config['F_IN']:,:].to_numpy())
    graphs += [g]

## Train-Test Split

In [44]:
splits = (0.6, 0.1, 0.3)
split_train, split_val, _ = splits
index_train = int(np.floor(config["N_GRAPHS"]*split_train))
index_val = int(index_train + np.floor(config["N_GRAPHS"]*split_val))
train_g = graphs[:index_train]
val_g = graphs[index_train:index_val]
test_g = graphs[index_val:]

print("Size of train data:", len(train_g))
print("Size of validation data:", len(val_g))
print("Size of test data:", len(test_g))

Size of train data: 432
Size of validation data: 72
Size of test data: 216
